# Running an Individual Test
This notebook shows how to run individual metrics or thresholds tests.

## Before you begin

To access the [ValidMind Platform UI](https://app.prod.validmind.ai), you'll need an account. If you're new, you can create your account the first time you log in. Signing up is FREE.

If you encounter errors due to missing modules in your Python environment, install the modules with `pip install`, and then re-run the notebook. For more help, refer to [Installing Python Modules](https://docs.python.org/3/installing/index.html).

## Install the client library

In [1]:
%pip install -q validmind

## Initialize the client library

Every documentation project in the Platform UI comes with a _code snippet_ that lets the client library associate your documentation and tests with the right project on the Platform UI when you run this notebook. As you will see later, documentation projects are useful because they act as containers for model documentation and validation reports and they enable you to organize all of your documentation work in one place. 

Get your code snippet by creating a documentation project:

1. In a browser, log into the [Platform UI](https://app.prod.validmind.ai).

2. Go to **Documentation Projects** and click **Create new project**.

3. Select **`[Demo] Customer Churn Model`** and **`Initial Validation`** for the model name and type, give the project a unique  name to make it yours, and then click **Create project**.

4. Go to **Documentation Projects** > **YOUR_UNIQUE_PROJECT_NAME** > **Getting Started** and click **Copy snippet to clipboard**.

Next, replace this placeholder with your own code snippet:

In [2]:
## Replace with code snippet from your documentation project ##

import validmind as vm

vm.init(
  api_host = "https://api.prod.validmind.ai/api/v1/tracking",
  api_key = "...",
  api_secret = "...",
  project = "..."
)

2023-10-05 00:20:47,595 - INFO(validmind.api_client): Connected to ValidMind. Project: Customer Churn Model - Initial Validation v2 (cln9pes5900010wryh94c4q5j)


In [3]:
import pandas as pd
import xgboost as xgb

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

%matplotlib inline

## Load the Demo Dataset

In [4]:
# You can also import customer_churn like this:
from validmind.datasets.classification import customer_churn as demo_dataset
# from validmind.datasets.classification import taiwan_credit as demo_dataset

df = demo_dataset.load_data()

### Prepocess the Raw Dataset

In [5]:
train_df, validation_df, test_df = demo_dataset.preprocess(df)

## Train a Model for Testing

We train a simple customer churn model for our test.

In [6]:
x_train = train_df.drop(demo_dataset.target_column, axis=1)
y_train = train_df[demo_dataset.target_column]
x_val = validation_df.drop(demo_dataset.target_column, axis=1)
y_val = validation_df[demo_dataset.target_column]

model = xgb.XGBClassifier(early_stopping_rounds=10)
model.set_params(
    eval_metric=["error", "logloss", "auc"],
)
model.fit(
    x_train,
    y_train,
    eval_set=[(x_val, y_val)],
    verbose=False,
)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=10,
              enable_categorical=False, eval_metric=['error', 'logloss', 'auc'],
              feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=None, ...)

## Setup Test Inputs and Run the Test

### Initialize ValidMind objects

In [7]:
vm_train_ds = vm.init_dataset(
    dataset=train_df,
    type="generic",
    target_column=demo_dataset.target_column
)

vm_test_ds = vm.init_dataset(
    dataset=test_df,
    type="generic",
    target_column=demo_dataset.target_column
)

vm_model = vm.init_model(
    model,
    train_ds=vm_train_ds,
    test_ds=vm_test_ds,
)

2023-10-05 00:20:47,829 - INFO(validmind.client): The 'type' argument to init_dataset() argument is deprecated and no longer required.
2023-10-05 00:20:47,830 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
2023-10-05 00:20:47,853 - INFO(validmind.client): The 'type' argument to init_dataset() argument is deprecated and no longer required.
2023-10-05 00:20:47,853 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...


### Run the test

Individual tests can be easily run by calling the `run_test` function provided by the `validmind.tests` module. The function takes the following arguments:

- `test_id`: The ID of the test to run. To find a particular test and get its, refer to the `explore_tests.ipynb` notebook
- `params`: A dictionary of parameters for the test. These will override any `default_params` set in the test definition. Refer to the `explore_tests.ipynb` notebook to find the default parameters for a test.

You can then pass in any inputs for the test as keyword arguments. Most likely, these will be `dataset` and `model` objects. Again, you may refer to the `explore_tests.ipynb` notebook to find the required inputs for a test.

In [8]:
test = vm.tests.run_test(
    test_id="validmind.model_validation.sklearn.TrainingTestDegradation",
    params={}, # can be used to set overrides to the test's default parameters
    model=vm_model,
)

### Log the Test Result to ValidMind

Once the test has been run, you can save the results to the ValidMind platform by calling the `log` method of the test object returned after running the test.

In [9]:
test.log()